<a href="https://colab.research.google.com/github/rakshitraj/fchollet/blob/new-branch/4_Scalar%20Regression-Boston%20housing%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scalar Regression

Design a neural network to predict the median prices of homes in a give Boston suburb in the mid-1970's, using the Python library Keras.

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

## About the dataset

The Boston data frame has 506 rows and 14 columns.

This data frame contains the following columns:

`crim` : 
per capita crime rate by town.

`zn` : 
proportion of residential land zoned for lots over 25,000 sq.ft.

`indus` : 
proportion of non-retail business acres per town.

`chas` : 
Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

`nox` : 
nitrogen oxides concentration (parts per 10 million).

`rm` : 
average number of rooms per dwelling.

`age` : 
proportion of owner-occupied units built prior to 1940.

`dis` : 
weighted mean of distances to five Boston employment centres.

`rad` : 
index of accessibility to radial highways.

`tax` : 
full-value property-tax rate per \$10,000.

`ptratio` : 
pupil-teacher ratio by town.

`black` : 
1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

`lstat` : 
lower status of the population (percent).

`medv` : 
median value of owner-occupied homes in \$1000s.

Source
Harrison, D. and Rubinfeld, D.L. (1978) Hedonic prices and the demand for clean air. J. Environ. Economics and Management 5, 81–102.

Belsley D.A., Kuh, E. and Welsch, R.E. (1980) Regression Diagnostics. Identifying Influential Data and Sources of Collinearity. New York: Wiley.

It has relatively few data points: only 506, split
between 404 training samples and 102 test samples. And each feature in the input data (for example, the crime rate) has a different scale.

## Loading the dataset

In [2]:
from keras.datasets import boston_housing

(train_data, train_targets),(test_data, test_targets) = boston_housing.load_data() 

57344/57026 [==============================] - 0s 7us/step


In [3]:
train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

In [5]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

Targets are the median values of owner-occupied homes, in thousands of dollars.

## Preparing the data

Every feature of the dataset takes on very different ranges. While a neural netwok will certainly adapt to such _hetrogeneous_ data, it would make learning more difficult. To deal with this we shall implement feature-wise normalization: _for each feature in the input data, subtract the mean of the feature and divide by standard deviation, so that the feature is centered around 0 with unit standard deviation_. 

We will do this using Numpy.

In [6]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

# Normalizing the testing data
test_data -= mean
test_data /= std

## Building our network

In [7]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, 
                           activation='relu', 
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    # Compile model
    model.compile(optimizer='rmsprop', 
                  loss='mse', 
                  metrics=['mae'])
    return model

## Setting up K-fold validation

In [8]:
import numpy as np

k = 4
num_val_samples = len(train_data)//k
num_epochs = 100
all_scores = []

In [9]:
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],
                                        train_data[(i+1) * num_val_samples:]],
                                       axis=0)
    partial_train_target = np.concatenate([train_targets[:i * num_val_samples],
                                        train_targets[(i+1) * num_val_samples:]],
                                       axis=0)
    model = build_model()
    model.fit(partial_train_data,
              partial_train_target,
              epochs=num_epochs,
              batch_size = 1,
              verbose=1)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
    all_scores.append(val_mae)

processing fold # 0
Epoch 1/100
303/303 [==============================] - 2s 8ms/step - loss: 201.4799 - mae: 10.7849
Epoch 2/100
303/303 [==============================] - 1s 3ms/step - loss: 32.0280 - mae: 3.9814
Epoch 3/100
303/303 [==============================] - 1s 3ms/step - loss: 21.5678 - mae: 3.1097
Epoch 4/100
303/303 [==============================] - 1s 3ms/step - loss: 18.5455 - mae: 2.8488
Epoch 5/100
303/303 [==============================] - 1s 3ms/step - loss: 16.4418 - mae: 2.6822
Epoch 6/100
303/303 [==============================] - 1s 3ms/step - loss: 14.8145 - mae: 2.5420
Epoch 7/100
303/303 [==============================] - 1s 3ms/step - loss: 14.1898 - mae: 2.4393
Epoch 8/100
303/303 [==============================] - 1s 3ms/step - loss: 13.3019 - mae: 2.4636
Epoch 9/100
303/303 [==============================] - 1s 3ms/step - loss: 12.6402 - mae: 2.2873
Epoch 10/100
303/303 [==============================] - 1s 3ms/step - loss: 12.0970 - mae: 2.3680
Epoch 1

In [11]:
all_scores

[2.201504707336426, 2.230896472930908, 2.8226723670959473, 2.6316893100738525]

In [10]:
np.mean(all_scores)

2.4716907143592834

## Saving validation logs at each fold

In [27]:
num_epochs = 500
all_mae_histories = []
for i in range (k):
  print ('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
  
  partial_train_data = np.concatenate([train_data[:i*num_val_samples], 
                                     train_data[(i+1)*num_val_samples:]],
                                     axis=0)
  
  partial_train_targets = np.concatenate([train_targets[:i*num_val_samples], 
                                     train_targets[(i+1)*num_val_samples:]],
                                     axis=0)
  
  model = build_model()

  history = model.fit(partial_train_data,
                      partial_train_targets,
                      epochs= num_epochs,
                      batch_size = 1, verbose =0, workers=4 ,
                      validation_data = (val_data,val_targets))
  mae_history = history.history['val_mae']
  all_mae_histories.append(mae_history)
  

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [24]:
history.history.keys()

dict_keys(['loss', 'mae'])

In [25]:
mae_history = history.history['mae']